In [28]:
import json

import requests
import pandas as pd
import geopandas as gpd
from elasticsearch import Elasticsearch

In [4]:
normative_df = pd.read_parquet("normative_data_cleaned.parquet")

In [5]:
ec = Elasticsearch(hosts=[f"http://localhost:9300"])

In [6]:
body={
    "mappings": {
        "properties": {
            "body_vector": {
                "type": "dense_vector",
                "dims": 4096,
                "index": True,
                "similarity": "cosine",
            },
            "body": {"type": "text"},
            "num_id": {"type": "long"},
            "feature_collection": {      # ← новое поле
                "type": "object",
                "enabled": True
            }
        }
    }
}

ec.indices.create(
    index="1830&general",
    body=body
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': '1830&general'})

In [2]:
from iduconfig import Config
from src.vectorizer.vectorizer_service import VectorizerService

config = Config()
vectorized = VectorizerService(config)

ValueError: APP_ENV variable is not present

In [9]:
import requests
import pandas as pd
import geopandas as gpd


all_buffers_resp = requests.get(
    "api/v1/scenarios/1830/buffers",
    headers=headers
)

In [10]:
from shapely.geometry import shape

df = pd.json_normalize(all_buffers_resp.json()["features"], sep="_")
df["geometry"] = df.apply(lambda x: shape({"type": x["geometry_type"], "coordinates": x["geometry_coordinates"]}), axis=1)
gdf = gpd.GeoDataFrame(df.drop(columns=["geometry_type", "geometry_coordinates", "type", "properties_is_scenario_object", "properties_urban_object_object_geometry_territory_name", "properties_urban_object_object_geometry_territory_id"]).rename(columns={"properties_urban_object_object_geometry_id": "geometry_id", "properties_urban_object_physical_object_type_name": "physical_object_type", "properties_urban_object_physical_object_type_id": "object_type_id", "properties_urban_object_physical_object_name": "physical_object_name", "properties_buffer_type_description": "description", "properties_buffer_type_name": "buffer_type-name"}), geometry="geometry", crs=4326)

In [11]:
gdf["zones_area"] = gdf.to_crs(gdf.estimate_utm_crs()).area.apply(lambda x: round(x, 2))

In [67]:
def aggr_geometry(x)

,properties_buffer_type_id,buffer_type-name,description,properties_urban_object_id,properties_urban_object_physical_object_id,physical_object_name,object_type_id,physical_object_type,geometry_id,properties_urban_object_service,properties_is_custom,properties_is_locked,geometry,zones_area
0,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970031,1473528,None,52,Местная дорога,1149608,None,False,False,"POLYGON ((30.73785 60.05051, 30.73784 60.0505,...",1911.06
1,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970037,1473534,None,52,Местная дорога,1149614,None,False,False,"POLYGON ((30.72326 60.04667, 30.72325 60.04667...",5677.74
2,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970043,1473540,None,52,Местная дорога,1149620,None,False,False,"POLYGON ((30.72559 60.04915, 30.72559 60.04914...",347.56
3,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970049,1473546,None,52,Местная дорога,1149626,None,False,False,"POLYGON ((30.72658 60.04947, 30.72658 60.04947...",337.58
4,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970055,1473552,None,52,Местная дорога,1149632,None,False,False,"POLYGON ((30.72881 60.05021, 30.72881 60.05021...",327.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970193,1473690,None,52,Местная дорога,1149770,None,False,False,"POLYGON ((30.71541 60.03899, 30.71541 60.039, ...",718.05
170,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970199,1473696,None,52,Местная дорога,1149776,None,False,False,"POLYGON ((30.71336 60.03939, 30.71336 60.0394,...",340.85
171,4,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,1970976,1474584,None,43,Промышленная территория,1150523,None,False,False,"POLYGON ((30.71274 60.03764, 30.71143 60.03862...",1108194.47
172,5,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,1970195,1473692,None,52,Местная дорога,1149772,None,False,False,"POLYGON ((30.71352 60.03785, 30.71351 60.03785...",466.06


In [30]:
common_info = {}

In [69]:
def collect_featurecollection(group):
    """
    Превращает группу gdf в FeatureCollection.
    Вытаскивает все колонки (и geometry) как properties.
    """
    # GeoPandas → GeoJSON → Python dict
    geojson = json.loads(group.to_json())

    return {
        "type": "FeatureCollection",
        "features": geojson["features"]
    }

In [81]:
zones_info_gdf = (
    gdf
    .groupby("buffer_type-name")
    .agg(
        description=("description", "first"),
        zones_area=("zones_area", "sum"),
        feature_collection=("geometry", lambda _: None)  # placeholder
    )
)
zones_info_gdf["feature_collection"] = (
    gdf.groupby("buffer_type-name").apply(collect_featurecollection)
)
zones_info_gdf["zones_area"] = (zones_info_gdf["zones_area"] / 1e6).round(2).astype(str) + " кв.км"
common_info["zones"] = zones_info_gdf.reset_index().to_dict(orient="records")

C:\Users\thebe\AppData\Local\Temp\ipykernel_21140\1954482858.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gdf.groupby("buffer_type-name").apply(collect_featurecollection)


In [82]:
common_info["zones"]

[{'buffer_type-name': 'Придорожная полоса автомобильных дорог',
  'description': 'В границах полосы отвода автомобильной дороги федерального значения запрещаются: - выполнение работ, не связанных со строительством, с реконструкцией, капитальным ремонтом, ремонтом и содержанием автомобильной дороги, а также с размещением объектов дорожного сервиса - размещение зданий, строений, сооружений и других объектов, не предназначенных для обслуживания автомобильной дороги, ее строительства, реконструкции, капитального ремонта, ремонта и содержания и не относящихся к объектам дорожного сервиса; - распашка земельных участков, покос травы, осуществление рубок и повреждение лесных насаждений и иных многолетних насаждений, снятие дерна и выемка грунта, за исключением работ по содержанию полосы отвода автомобильной дороги или ремонту автомобильной дороги, ее участков; - выпас животных, а также их прогон через автомобильные дороги вне специально установленных мест, согласованных с владельцами автомобил

In [83]:
common_info["zones"] = [
    {
        "text": str(
            {
                "buffer_type-name": i["buffer_type-name"],
                "description": i["description"],
                "zones_area": i["zones_area"]
            }
        ),
        "feature_collection": i["feature_collection"],
    } for i in common_info["zones"]
]

In [33]:
proj_indicators_resp = requests.get(
    "api/v1/scenarios/1830/indicators_values",
    headers=headers
)

proj_indicators = pd.json_normalize(proj_indicators_resp.json(), sep="&").drop(columns=["territory", "hexagon_id", "information_source", "indicator&list_label", "scenario&id", "indicator&measurement_unit", "scenario&name", "indicator&level", "indicator&measurement_unit&id", "indicator&parent_id", "updated_at", "created_at", "indicator_value_id"])
proj_indicators["value"] = proj_indicators["value"].astype(str) + " " + proj_indicators["indicator&measurement_unit&name"].fillna("балл").astype(str)
proj_indicators.drop(columns=["indicator&measurement_unit&name"], inplace=True)
proj_indicators["formed_context"] = proj_indicators["indicator&name_full"] +", " + "Значение показателя: " + proj_indicators["value"] + ", Комментарий: " + proj_indicators["comment"].fillna("--")

common_info["indicators_info"] = proj_indicators[["formed_context"]].to_dict(orient="records")

In [104]:
all_objects = requests.get(
    "/api/v1/scenarios/1830/geometries_with_all_objects",
    headers=headers
)
all_objects_gdf = gpd.GeoDataFrame.from_features(all_objects.json(), crs=4326)
all_objects_gdf.drop(columns=["territory", "object_geometry_id", "osm_id"], inplace=True)

In [105]:
def parse_type_name(data: list[dict] | None) -> str | list[str] | None:
    if data:
        if len(data) == 1:
            return data[0]["physical_object_type"]["name"]
        else:
            return [data[i]["physical_object_type"]["name"] for i in range(len(data))]
    else:
        return None

all_objects_gdf["physical_object_type_name"] = all_objects_gdf["physical_objects"].apply(parse_type_name)

In [117]:
def parse_object_name(data: list[dict] | None) -> str | list[str] | None:

    if data:
        if len(data) == 1:
            return data[0]["name"]
        else:
            return [data[i]["name"] for i in range(len(data))]
    else:
        return None

all_objects_gdf["object_name"] = all_objects_gdf["physical_objects"].apply(parse_object_name)

In [126]:
def parse_floors(data: list[dict] | None) -> list[str] | list[str] | None:

    if data:
        if len(data) == 1:
            if building_data:=data[0].get("building"):
                return building_data.get("floors")
            else:
                return None
        else:
            res_list = []
            for i in range(len(data)):
                if data[i]:
                    if data[i].get("building"):
                         res_list.append(data[i]["building"].get("name"))
                    else:
                        res_list.append(None)
                else:
                    res_list.append(None)
            return res_list
    else:
        return None

all_objects_gdf["floors"] = all_objects_gdf["physical_objects"].apply(parse_floors)

In [130]:
all_objects_gdf.drop(columns=["physical_objects", "is_scenario_object", "is_locked"], inplace=True)

In [137]:
all_objects_gdf["Площадь фундамента"] = all_objects_gdf.to_crs(all_objects_gdf.estimate_utm_crs()).area.round(2)
all_objects_gdf["area"] = (all_objects_gdf["Площадь фундамента"] * all_objects_gdf["floors"]).round(2)

In [138]:
all_objects_gdf[all_objects_gdf["area"].notna()]

,geometry,address,services,physical_object_type_name,object_name,floors,Площадь фундамента,objects_num,area
173,"POLYGON ((30.725 60.04121, 30.72533 60.0414, 3...",None,[],Жилой дом,None,5.0,1645.27,1,8226.35
174,"POLYGON ((30.72542 60.04308, 30.72517 60.0431,...",None,[],Жилой дом,None,7.0,882.18,1,6175.26
175,"POLYGON ((30.72483 60.04255, 30.7247 60.04264,...",None,[],Жилой дом,None,7.0,304.64,1,2132.48
176,"POLYGON ((30.73406 60.05145, 30.73397 60.05153...",None,"[{'service_id': 117290, 'service_type': {'id':...",Жилой дом,None,4.0,1667.05,1,6668.20
177,"POLYGON ((30.72343 60.04831, 30.72346 60.04833...",None,[],Жилой дом,None,3.0,625.11,1,1875.33
...,...,...,...,...,...,...,...,...,...
539,"POLYGON ((30.72876 60.04294, 30.72881 60.04293...",None,[],Нежилое здание,None,2.0,2776.68,1,5553.36
540,"POLYGON ((30.72794 60.04437, 30.72801 60.04427...",None,[],Нежилое здание,None,2.0,3920.02,1,7840.04
541,"POLYGON ((30.72482 60.04664, 30.72591 60.047, ...",None,[],Нежилое здание,None,2.0,2528.96,1,5057.92
542,"POLYGON ((30.72104 60.0377, 30.72065 60.03787,...",None,[],Нежилое здание,None,2.0,1511.98,1,3023.96


In [132]:
all_objects_gdf["objects_num"] = 1

In [145]:
objects_info_gdf = all_objects_gdf.groupby("physical_object_type_name").agg(
        object_name=("object_name", set),
        base_area=("Площадь фундамента", "sum"),
        area=("area", "sum"),
        mean_floors=("floors", "mean"),
        objects_nums=("objects_num", "sum"),
        feature_collection=("geometry", lambda _: None)  # placeholder
    )

objects_info_gdf["feature_collection"] = (
    all_objects_gdf.groupby("physical_object_type_name").apply(collect_featurecollection)
)

C:\Users\thebe\AppData\Local\Temp\ipykernel_21140\4278228156.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_objects_gdf.groupby("physical_object_type_name").apply(collect_featurecollection)


In [146]:
for i in ["base_area", "area", "mean_floors"]:
    objects_info_gdf[i] = objects_info_gdf[i].round(2).replace(0.0, None)
objects_info_gdf

,object_name,base_area,area,mean_floors,objects_nums,feature_collection
physical_object_type_name,,,,,,
Жилой дом,{None},91880.95,435543.13,2.94,322,"{'type': 'FeatureCollection', 'features': [{'i..."
Местная дорога,{None},None,None,NaN,172,"{'type': 'FeatureCollection', 'features': [{'i..."
Нежилое здание,"{None, (Безымянный физический объект)}",130701.07,242262.03,1.84,45,"{'type': 'FeatureCollection', 'features': [{'i..."
Промышленная территория,{None},397036.92,None,NaN,2,"{'type': 'FeatureCollection', 'features': [{'i..."
Рекреационная зона,{None},214716.1,None,NaN,3,"{'type': 'FeatureCollection', 'features': [{'i..."


In [150]:
objects_info_gdf.reset_index().to_dict(orient="records")

[{'physical_object_type_name': 'Жилой дом',
  'object_name': {None},
  'base_area': 91880.95,
  'area': 435543.13,
  'mean_floors': 2.94,
  'objects_nums': 322,
  'feature_collection': {'type': 'FeatureCollection',
   'features': [{'id': '173',
     'type': 'Feature',
     'properties': {'address': None,
      'services': [],
      'physical_object_type_name': 'Жилой дом',
      'object_name': None,
      'floors': 5.0,
      'Площадь фундамента': 1645.27,
      'objects_num': 1,
      'area': 8226.35},
     'geometry': {'type': 'Polygon',
      'coordinates': [[[30.72500225339397, 60.04120931957111],
        [30.725332784460345, 60.04139883189263],
        [30.725334807107036, 60.041397889169374],
        [30.725721358911162, 60.04161505170528],
        [30.725893713544323, 60.04154045854311],
        [30.726032218489774, 60.041315670049286],
        [30.725832380169987, 60.04128794941821],
        [30.725732215859846, 60.041463017669024],
        [30.725476680914472, 60.0413317641639

In [152]:
common_info["objects"] = objects_info_gdf.reset_index().to_dict(orient="records")

common_info["objects"] = [
    {
        "text": str(
            {
                "physical_object_type_name": i["physical_object_type_name"],
                "object_name": i["object_name"],
                "base_area": i["base_area"],
                "area": i["area"],
                "mean_floors": i["mean_floors"],
                "objects_nums": i["objects_nums"],
            }
        ),
        "feature_collection": i["feature_collection"],
    } for i in common_info["objects"]
]

In [153]:
with open("test_common_json_to_load.json", "w") as tf:
    json.dump(common_info, tf)